In [1]:
import cv2
import numpy as np

def make_coordinates(image, line_parameters):
    try:
        slope, intercept = line_parameters
        y1 = image.shape[0]
        y2 = int(y1 * (3 / 5))
        x1 = int((y1 - intercept) / slope)
        x2 = int((y2 - intercept) / slope)
        return np.array([x1, y1, x2, y2])
    except TypeError:
        return None

def average_slope_intercept(image, lines):
    left_fit = []
    right_fit = []
    if lines is None:
        return None, None
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        parameters = np.polyfit((x1, x2), (y1, y2), 1)
        slope = parameters[0]
        intercept = parameters[1]
        if slope < 0:
            left_fit.append((slope, intercept))
        else:
            right_fit.append((slope, intercept))
    left_fit_average = np.average(left_fit, axis=0) if left_fit else None
    right_fit_average = np.average(right_fit, axis=0) if right_fit else None
    left_line = make_coordinates(image, left_fit_average) if left_fit_average is not None else None
    right_line = make_coordinates(image, right_fit_average) if right_fit_average is not None else None
    return left_line, right_line

def canny(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    canny = cv2.Canny(blur, 50, 150)
    return canny

def region_of_interest(image):
    height = image.shape[0]
    polygons = np.array([[(200, height), (1100, height), (550, 250)]])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, polygons, 255)
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

def display_lines(image, lines, colors=((255, 0, 0), (0, 255, 0))):
    line_image = np.zeros_like(image)
    for i, line in enumerate(lines):
        if line is not None:
            x1, y1, x2, y2 = line
            cv2.line(line_image, (x1, y1), (x2, y2), colors[i], 10)
    return line_image

def add_lane_departure_warning(image, left_line, right_line):
    height, width, _ = image.shape
    lane_center = (left_line[2] + right_line[2]) // 2
    vehicle_center = width // 2
    offset = vehicle_center - lane_center
    cv2.putText(image, f'Offset: {offset:.2f} pixels', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    if abs(offset) > 100:
        cv2.putText(image, 'Lane Departure Warning!', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

def measure_curvature(left_line, right_line):
    if left_line is None or right_line is None:
        return None, None
    left_fit = np.polyfit((left_line[1], left_line[3]), (left_line[0], left_line[2]), 2)
    right_fit = np.polyfit((right_line[1], right_line[3]), (right_line[0], right_line[2]), 2)
    return left_fit, right_fit

def draw_curvature_info(image, left_fit, right_fit):
    y_eval = image.shape[0]
    left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    cv2.putText(image, f'Left Curvature: {left_curverad:.2f} m', (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(image, f'Right Curvature: {right_curverad:.2f} m', (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

def calculate_lane_width(left_line, right_line):
    if left_line is None or right_line is None:
        return None
    lane_width = abs(right_line[0] - left_line[0])
    return lane_width

def draw_lane_width_info(image, lane_width):
    if lane_width is not None:
        cv2.putText(image, f'Lane Width: {lane_width:.2f} pixels', (50, 250), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

def detect_lane_change(image, left_line, right_line, prev_left_line, prev_right_line):
    if prev_left_line is None or prev_right_line is None:
        return False
    lane_change_detected = (
        abs(left_line[0] - prev_left_line[0]) > 50 or
        abs(right_line[0] - prev_right_line[0]) > 50
    )
    if lane_change_detected:
        cv2.putText(image, 'Lane Change Detected!', (50, 300), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

def save_video_output(cap, frame, output_file='output.avi'):
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width, frame_height))
    return out

def detect_road_condition(left_fit, right_fit):
    if left_fit is None or right_fit is None:
        return None
    left_curverad = ((1 + (2*left_fit[0]*left_fit[2] + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*right_fit[2] + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    avg_curverad = (left_curverad + right_curverad) / 2
    if avg_curverad < 1000:
        return 'Curvy Road'
    else:
        return 'Straight Road'

def draw_road_condition_info(image, condition):
    if condition is not None:
        cv2.putText(image, f'Road Condition: {condition}', (50, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

cap = cv2.VideoCapture("test2.mp4")
out = None
prev_left_line, prev_right_line = None, None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    if out is None:
        out = save_video_output(cap, frame)
    
    canny_image = canny(frame)
    cropped_image = region_of_interest(canny_image)
    lines = cv2.HoughLinesP(cropped_image, 2, np.pi / 180, 100, np.array([]), minLineLength=40, maxLineGap=5)
    left_line, right_line = average_slope_intercept(frame, lines)
    lines_image = display_lines(frame, [left_line, right_line])
    combo_image = cv2.addWeighted(frame, 0.8, lines_image, 1, 1)
    
    if left_line is not None and right_line is not None:
        add_lane_departure_warning(combo_image, left_line, right_line)
        left_fit, right_fit = measure_curvature(left_line, right_line)
        if left_fit is not None and right_fit is not None:
            draw_curvature_info(combo_image, left_fit, right_fit)
            road_condition = detect_road_condition(left_fit, right_fit)
            draw_road_condition_info(combo_image, road_condition)
        lane_width = calculate_lane_width(left_line, right_line)
        draw_lane_width_info(combo_image, lane_width)
        detect_lane_change(combo_image, left_line, right_line, prev_left_line, prev_right_line)

        prev_left_line, prev_right_line = left_line, right_line

    cv2.imshow("result", combo_image)
    out.write(combo_image)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
if out:
    out.release()
cv2.destroyAllWindows()
